In [1]:
import glob, os
import re

In [2]:
glob_list = []

#loop through subject folders and glob
for subject in range(25):
    glob_list.append(glob.glob(str(subject) + "\\[0-9]*.csv"))
    
#function to flatten glob
flatten = lambda l: [item for sublist in l for item in sublist]
glob_list = flatten(glob_list)
glob_list

['0\\0_Left.csv',
 '0\\0_Right.csv',
 '0\\10_Left.csv',
 '0\\10_Right.csv',
 '0\\1_Left.csv',
 '0\\1_Right.csv',
 '0\\2_Left.csv',
 '0\\2_Right.csv',
 '0\\3_Left.csv',
 '0\\3_Right.csv',
 '0\\4_Left.csv',
 '0\\4_Right.csv',
 '0\\5_Left.csv',
 '0\\5_Right.csv',
 '0\\6_Left.csv',
 '0\\6_Right.csv',
 '0\\7_Left.csv',
 '0\\7_Right.csv',
 '0\\8_Left.csv',
 '0\\8_Right.csv',
 '0\\9_Left.csv',
 '0\\9_Right.csv',
 '1\\0_Left.csv',
 '1\\0_Right.csv',
 '1\\10_Left.csv',
 '1\\10_Right.csv',
 '1\\1_Left.csv',
 '1\\1_Right.csv',
 '1\\2_Left.csv',
 '1\\2_Right.csv',
 '1\\3_Left.csv',
 '1\\3_Right.csv',
 '1\\4_Left.csv',
 '1\\4_Right.csv',
 '1\\5_Left.csv',
 '1\\5_Right.csv',
 '1\\6_Left.csv',
 '1\\6_Right.csv',
 '1\\7_Left.csv',
 '1\\7_Right.csv',
 '1\\8_Left.csv',
 '1\\8_Right.csv',
 '1\\9_Left.csv',
 '1\\9_Right.csv',
 '2\\0_Left.csv',
 '2\\0_Right.csv',
 '2\\10_Left.csv',
 '2\\10_Right.csv',
 '2\\1_Left.csv',
 '2\\1_Right.csv',
 '2\\2_Left.csv',
 '2\\2_Right.csv',
 '2\\3_Left.csv',
 '2\\3_Right.c

In [3]:
#make a list of tuples of hand pairings
lr_pairs = list(zip(*[iter(glob_list)]*2))
lr_pairs

[('0\\0_Left.csv', '0\\0_Right.csv'),
 ('0\\10_Left.csv', '0\\10_Right.csv'),
 ('0\\1_Left.csv', '0\\1_Right.csv'),
 ('0\\2_Left.csv', '0\\2_Right.csv'),
 ('0\\3_Left.csv', '0\\3_Right.csv'),
 ('0\\4_Left.csv', '0\\4_Right.csv'),
 ('0\\5_Left.csv', '0\\5_Right.csv'),
 ('0\\6_Left.csv', '0\\6_Right.csv'),
 ('0\\7_Left.csv', '0\\7_Right.csv'),
 ('0\\8_Left.csv', '0\\8_Right.csv'),
 ('0\\9_Left.csv', '0\\9_Right.csv'),
 ('1\\0_Left.csv', '1\\0_Right.csv'),
 ('1\\10_Left.csv', '1\\10_Right.csv'),
 ('1\\1_Left.csv', '1\\1_Right.csv'),
 ('1\\2_Left.csv', '1\\2_Right.csv'),
 ('1\\3_Left.csv', '1\\3_Right.csv'),
 ('1\\4_Left.csv', '1\\4_Right.csv'),
 ('1\\5_Left.csv', '1\\5_Right.csv'),
 ('1\\6_Left.csv', '1\\6_Right.csv'),
 ('1\\7_Left.csv', '1\\7_Right.csv'),
 ('1\\8_Left.csv', '1\\8_Right.csv'),
 ('1\\9_Left.csv', '1\\9_Right.csv'),
 ('2\\0_Left.csv', '2\\0_Right.csv'),
 ('2\\10_Left.csv', '2\\10_Right.csv'),
 ('2\\1_Left.csv', '2\\1_Right.csv'),
 ('2\\2_Left.csv', '2\\2_Right.csv'),
 ('2\\

In [4]:
import pandas as pd

df_list = []

for pair in lr_pairs:
    df_left = pd.read_csv(pair[0], index_col=None).drop(['Unnamed: 0'], axis = 1)
    df_right = pd.read_csv(pair[1], index_col=None).drop(['Unnamed: 0'], axis = 1)
    
    #rename columns
    df_left = df_left.add_prefix('left')
    df_right = df_right.add_prefix('right')
    
    #merge
    df = pd.merge(df_left, df_right, left_on='leftTime', right_on='rightTime').drop('rightTime', axis = 1)

    #covert fist column to time object
    df['leftTime'] = pd.to_datetime(df['leftTime'].str[:-3], format = '%H:%M:%S.%f')
    
    #difference between rows
    df = df.diff().iloc[1:]
    df['leftTime'] = df['leftTime'].dt.total_seconds()
    
    df.rename(columns={'leftTime':'time'}, inplace=True)
    
    #add sign and subject using regex of file name
    subject_sign = re.split(r'\\', re.findall('^[^_]+(?=_)', pair[0])[0])
    df.insert(loc = 0, column = 'Subject', value = subject_sign[0])
    df.insert(loc = 0, column = 'Sign', value = subject_sign[1])
    
#this makes velocity columns

#     for col in df.iloc[:,2:]:
#         df[col + ' velocity'] = df[col]/df['Time']
    
    df_list.append(df)

In [5]:
#numbers 0-10 for 25 subjects
len(df_list)

275

In [6]:
#example entry
df_list[35]

,Sign,Subject,time,left thumbProximal_L_X,left thumbProximal_L_Y,left thumbProximal_L_Z,left thumbDistal_L_X,left thumbDistal_L_Y,left thumbDistal_L_Z,left thumbEF_L_X,...,right armFrontRadius_L_Z,right armFrontUlna_L_X,right armFrontUlna_L_Y,right armFrontUlna_L_Z,right armBackLateral_L_X,right armBackLateral_L_Y,right armBackLateral_L_Z,right armBackMedial_L_X,right armBackMedial_L_Y,right armBackMedial_L_Z
1,1,3,0.024,-0.000577,0.000184,-0.000018,-0.000519,7.800000e-07,0.000178,-0.000377,...,0.000050,-0.000026,0.000057,3.140000e-05,0.000094,0.000116,0.000057,0.000044,0.000091,0.000038
2,1,3,0.020,-0.000593,0.000440,-0.000814,-0.000707,7.145700e-04,-0.001065,-0.000806,...,0.000021,-0.000063,0.000028,4.000000e-07,-0.000055,0.000010,0.000045,-0.000122,-0.000010,0.000025
3,1,3,0.029,-0.000376,0.000027,-0.000712,-0.000767,7.511100e-04,-0.001306,-0.001038,...,-0.000062,-0.000336,-0.000227,-4.470000e-05,-0.000235,-0.000281,0.000181,-0.000228,-0.000239,0.000198
4,1,3,0.032,-0.001070,0.000597,-0.001343,-0.001094,1.048740e-03,-0.002128,-0.000922,...,0.000079,-0.000257,-0.000123,1.661000e-04,-0.000370,-0.000337,0.000453,-0.000194,-0.000195,0.000541
5,1,3,0.028,-0.000710,0.000449,-0.001115,-0.000682,8.448700e-04,-0.001965,-0.000504,...,-0.000016,-0.000379,-0.000141,-2.120000e-05,-0.000485,-0.000330,0.000225,-0.000563,-0.000306,0.000219
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
943,1,3,0.033,0.000195,-0.000052,0.000095,0.000155,-2.191000e-05,0.000122,0.000092,...,-0.004531,-0.001706,0.002034,-2.296600e-03,-0.001784,0.002714,-0.003123,-0.001075,0.001443,-0.000889
944,1,3,0.034,0.000081,0.000188,0.000163,0.000051,2.262000e-04,0.000168,0.000029,...,-0.006361,-0.000087,0.003888,-4.143400e-03,0.001410,0.007658,-0.005938,0.002680,0.006026,-0.003721
945,1,3,0.031,-0.000011,0.000132,0.000007,0.000041,5.290000e-05,0.000011,0.000083,...,-0.008012,0.000527,0.003396,-5.145200e-03,0.002316,0.005331,-0.004586,0.003903,0.003760,-0.001718
946,1,3,0.034,-0.000034,0.000129,-0.000044,0.000028,4.935000e-05,-0.000056,0.000082,...,-0.010998,0.001088,0.005492,-8.365500e-03,-0.002987,-0.001899,-0.006441,-0.002936,-0.001600,-0.003809


In [7]:
#concatenate list of frames
sign_frame = pd.concat(df_list, axis=0, ignore_index=False)
sign_frame

,Sign,Subject,time,left thumbProximal_L_X,left thumbProximal_L_Y,left thumbProximal_L_Z,left thumbDistal_L_X,left thumbDistal_L_Y,left thumbDistal_L_Z,left thumbEF_L_X,...,right armFrontRadius_L_Z,right armFrontUlna_L_X,right armFrontUlna_L_Y,right armFrontUlna_L_Z,right armBackLateral_L_X,right armBackLateral_L_Y,right armBackLateral_L_Z,right armBackMedial_L_X,right armBackMedial_L_Y,right armBackMedial_L_Z
1,0,0,0.028,0.000388,0.000133,0.000050,0.000406,0.000153,0.000063,0.000403,...,-0.000321,-0.000166,0.000114,-0.000298,-8.360000e-05,-0.000393,-0.000142,-0.000071,-0.000331,-0.000119
2,0,0,0.036,0.000420,0.000077,-0.000248,0.000443,0.000085,-0.000239,0.000429,...,-0.000235,-0.000095,0.000121,-0.000193,4.000000e-05,-0.000276,-0.000075,0.000035,-0.000261,-0.000034
3,0,0,0.033,0.000223,0.000264,-0.000160,0.000164,0.000275,-0.000170,0.000121,...,-0.000251,-0.000084,0.000163,-0.000204,-1.250000e-04,-0.000481,0.000008,-0.000147,-0.000522,0.000054
4,0,0,0.034,0.000103,0.000078,-0.000318,0.000105,0.000103,-0.000307,0.000054,...,-0.000213,-0.000179,0.000077,-0.000056,1.000000e-07,-0.000630,0.000164,-0.000104,-0.000869,0.000321
5,0,0,0.036,0.000055,0.000233,-0.000030,0.000002,0.000263,-0.000031,-0.000022,...,-0.000001,0.000016,0.000356,0.000150,5.750000e-04,0.000152,0.000179,0.000504,0.000025,0.000330
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
787,9,24,0.038,-0.000011,-0.000063,0.000002,0.000031,-0.000226,0.000005,0.000049,...,-0.000274,0.000073,-0.000148,0.000050,7.976000e-04,-0.000154,0.000471,0.000318,-0.000516,0.000796
788,9,24,0.038,-0.000111,-0.000074,0.000067,-0.000147,0.000071,0.000060,-0.000166,...,-0.000076,0.000082,-0.000026,0.000101,8.564000e-04,0.000209,0.000298,0.000595,0.000053,0.000475
789,9,24,0.032,-0.000067,-0.000127,0.000220,-0.000082,-0.000164,0.000270,-0.000127,...,-0.000215,-0.000148,0.000127,0.000050,4.664000e-04,0.000229,0.000299,0.000041,-0.000108,0.000563
790,9,24,0.042,-0.000178,0.000430,-0.000121,-0.000261,0.000681,-0.000094,-0.000290,...,-0.000894,-0.000068,0.000466,-0.000368,1.420000e-04,0.000336,-0.000105,-0.000832,-0.000595,0.000422


# Let's write a class for feature extraction:

In [8]:
pairs = pd.read_csv('pairs.csv')
pairs = list(zip(pairs['0'], pairs['1']))

In [9]:
from itertools import combinations
from sklearn import linear_model

class extraction:
    def __init__(self, df):
        self.df = df
        self.features = dict()
        
    def label(self):
        self.features['label'] = self.df['Sign'].iloc[0]
        self.df = self.df.iloc[:, 2:]
        
    def mean(self):
        for col in self.df:
            self.features[col + ' mean'] = self.df[col].mean()
            
    def stdev(self):
        for col in self.df:
            self.features[col + ' stdev'] = self.df[col].std()
    
    def linear(self):
        #use this for all combinations
        cc = list(combinations(self.df.columns,2))
        
        for pair in pairs:
            try:
                regr = linear_model.LinearRegression()
                regr.fit(self.df[[pair[0]]], self.df[[pair[1]]])
                
                self.features[(pair[0], pair[1],'weight')] = regr.coef_[0][0] 
                
                r2 = regr.score(regr.predict(self.df[[pair[0]]]), self.df[[pair[1]]])
                self.features[(pair[0],pair[1],'R2')] = r2 
            except:
                continue
            
            
    def extract_features(self):
        self.label()
        self.mean()
        self.stdev()
        self.linear()

###########################################################################################

# Example:

In [10]:
df_list[0]

,Sign,Subject,time,left thumbProximal_L_X,left thumbProximal_L_Y,left thumbProximal_L_Z,left thumbDistal_L_X,left thumbDistal_L_Y,left thumbDistal_L_Z,left thumbEF_L_X,...,right armFrontRadius_L_Z,right armFrontUlna_L_X,right armFrontUlna_L_Y,right armFrontUlna_L_Z,right armBackLateral_L_X,right armBackLateral_L_Y,right armBackLateral_L_Z,right armBackMedial_L_X,right armBackMedial_L_Y,right armBackMedial_L_Z
1,0,0,0.028,0.000388,0.000133,0.000050,0.000406,0.000153,0.000063,0.000403,...,-0.000321,-0.000166,0.000114,-0.000298,-8.360000e-05,-0.000393,-0.000142,-0.000071,-0.000331,-0.000119
2,0,0,0.036,0.000420,0.000077,-0.000248,0.000443,0.000085,-0.000239,0.000429,...,-0.000235,-0.000095,0.000121,-0.000193,4.000000e-05,-0.000276,-0.000075,0.000035,-0.000261,-0.000034
3,0,0,0.033,0.000223,0.000264,-0.000160,0.000164,0.000275,-0.000170,0.000121,...,-0.000251,-0.000084,0.000163,-0.000204,-1.250000e-04,-0.000481,0.000008,-0.000147,-0.000522,0.000054
4,0,0,0.034,0.000103,0.000078,-0.000318,0.000105,0.000103,-0.000307,0.000054,...,-0.000213,-0.000179,0.000077,-0.000056,1.000000e-07,-0.000630,0.000164,-0.000104,-0.000869,0.000321
5,0,0,0.036,0.000055,0.000233,-0.000030,0.000002,0.000263,-0.000031,-0.000022,...,-0.000001,0.000016,0.000356,0.000150,5.750000e-04,0.000152,0.000179,0.000504,0.000025,0.000330
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,0,0,0.042,0.000234,0.000309,-0.000214,0.000329,0.000273,-0.000204,0.000371,...,0.000295,0.001436,0.000441,0.000373,-8.165000e-04,-0.000436,-0.000397,-0.000785,-0.000404,-0.000319
686,0,0,0.046,0.000256,0.000157,-0.000107,0.000251,0.000190,-0.000097,0.000228,...,0.000138,0.002025,0.000737,0.000454,-1.395300e-03,-0.001677,-0.000466,-0.001304,-0.001543,-0.000149
687,0,0,0.043,0.000210,0.000039,-0.000039,0.000226,0.000065,-0.000026,0.000216,...,-0.000251,0.002495,0.001583,-0.000102,-1.360000e-03,-0.000036,-0.001245,-0.001321,0.000028,-0.001097
688,0,0,0.059,-0.000083,0.000201,0.000012,-0.000046,0.000252,0.000038,-0.000039,...,-0.000735,0.005017,0.002821,0.000149,-2.756600e-03,-0.002480,-0.001475,-0.001811,-0.002021,-0.000591


In [11]:
class_example = extraction(df_list[0])

In [12]:
%%time
class_example.extract_features()

Wall time: 4.03 s


In [13]:
class_example.features

{'label': '0',
 'time mean': 0.0435732946298984,
 'left thumbProximal_L_X mean': 9.093222060957905e-06,
 'left thumbProximal_L_Y mean': 1.3063599419448466e-05,
 'left thumbProximal_L_Z mean': -3.5371552975326123e-06,
 'left thumbDistal_L_X mean': 9.338316400580548e-06,
 'left thumbDistal_L_Y mean': 1.5072554426705366e-05,
 'left thumbDistal_L_Z mean': -2.7464441219158395e-06,
 'left thumbEF_L_X mean': 1.0285050798258355e-05,
 'left thumbEF_L_Y mean': 1.5039419448476045e-05,
 'left thumbEF_L_Z mean': -1.7150943396226527e-06,
 'left indexProximal_L_X mean': 4.504934687953546e-06,
 'left indexProximal_L_Y mean': 1.4902699564586361e-05,
 'left indexProximal_L_Z mean': -7.605805515239526e-06,
 'left indexMedial_L_X mean': 6.797822931785194e-06,
 'left indexMedial_L_Y mean': 1.6415341074020313e-05,
 'left indexMedial_L_Z mean': -5.068940493468809e-06,
 'left indexDistal_L_X mean': 9.89927431059506e-06,
 'left indexDistal_L_Y mean': 1.6352380261248164e-05,
 'left indexDistal_L_Z mean': -5.336

In [14]:
%matplotlib inline
import matplotlib.pylab as plt
import numpy as np

########################################################################################

# Run this for list. I picked an arbitarary sign

In [15]:
# R2s = [(key[:2], value) for key, value in class_example.features.items() if ('R2' in key and value > .5)]
# R2s = [item[0] for item in R2s]

In [16]:
# pd.DataFrame(R2s).to_csv('pairs.csv', index = False)

In [17]:
# plt.hist(R2s)

In [18]:
# greater_than = [True if x > .5 else False for x in R2s]

In [19]:
# sum(greater_than)

In [20]:
# sum(greater_than) / len(R2s)

#####################################################################################

## Make list of dicts to dataframe:

In [21]:
#this removes left hand
for idx, df in enumerate(df_list):
    df_list[idx] = df.loc[:,~df.columns.str.startswith('left')]

In [22]:
df_list[0]

,Sign,Subject,time,right thumbProximal_L_X,right thumbProximal_L_Y,right thumbProximal_L_Z,right thumbDistal_L_X,right thumbDistal_L_Y,right thumbDistal_L_Z,right thumbEF_L_X,...,right armFrontRadius_L_Z,right armFrontUlna_L_X,right armFrontUlna_L_Y,right armFrontUlna_L_Z,right armBackLateral_L_X,right armBackLateral_L_Y,right armBackLateral_L_Z,right armBackMedial_L_X,right armBackMedial_L_Y,right armBackMedial_L_Z
1,0,0,0.028,-0.000100,0.000042,-0.000299,-0.000078,0.000039,-3.071000e-04,-0.000069,...,-0.000321,-0.000166,0.000114,-0.000298,-8.360000e-05,-0.000393,-0.000142,-0.000071,-0.000331,-0.000119
2,0,0,0.036,-0.000002,0.000199,-0.000187,0.000015,0.000225,-1.838000e-04,0.000015,...,-0.000235,-0.000095,0.000121,-0.000193,4.000000e-05,-0.000276,-0.000075,0.000035,-0.000261,-0.000034
3,0,0,0.033,-0.000035,0.000296,-0.000250,-0.000073,0.000296,-2.370000e-04,-0.000106,...,-0.000251,-0.000084,0.000163,-0.000204,-1.250000e-04,-0.000481,0.000008,-0.000147,-0.000522,0.000054
4,0,0,0.034,-0.000138,0.000626,-0.000216,-0.000229,0.000609,-1.911000e-04,-0.000309,...,-0.000213,-0.000179,0.000077,-0.000056,1.000000e-07,-0.000630,0.000164,-0.000104,-0.000869,0.000321
5,0,0,0.036,-0.000138,0.000650,-0.000026,-0.000236,0.000633,7.000000e-07,-0.000279,...,-0.000001,0.000016,0.000356,0.000150,5.750000e-04,0.000152,0.000179,0.000504,0.000025,0.000330
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,0,0,0.042,0.001564,0.000210,0.000265,0.001507,-0.000664,5.004000e-04,0.001807,...,0.000295,0.001436,0.000441,0.000373,-8.165000e-04,-0.000436,-0.000397,-0.000785,-0.000404,-0.000319
686,0,0,0.046,0.001785,-0.000203,0.000642,0.001734,-0.001654,9.838000e-04,0.002163,...,0.000138,0.002025,0.000737,0.000454,-1.395300e-03,-0.001677,-0.000466,-0.001304,-0.001543,-0.000149
687,0,0,0.043,0.002699,0.000931,0.000509,0.002810,0.000097,8.319000e-04,0.003130,...,-0.000251,0.002495,0.001583,-0.000102,-1.360000e-03,-0.000036,-0.001245,-0.001321,0.000028,-0.001097
688,0,0,0.059,0.005148,0.001957,0.000299,0.005855,0.001178,1.313300e-03,0.006791,...,-0.000735,0.005017,0.002821,0.000149,-2.756600e-03,-0.002480,-0.001475,-0.001811,-0.002021,-0.000591


In [23]:
%%time
features = []

for df in df_list:
    class_obj = extraction(df)
    class_obj.extract_features()
    
    features.append(class_obj.features)

Wall time: 8min 9s


In [24]:
feature_frame = pd.DataFrame(features)

In [25]:
feature_frame

,label,time mean,right thumbProximal_L_X mean,right thumbProximal_L_Y mean,right thumbProximal_L_Z mean,right thumbDistal_L_X mean,right thumbDistal_L_Y mean,right thumbDistal_L_Z mean,right thumbEF_L_X mean,right thumbEF_L_Y mean,...,"(right elbow_L_Z, right armBackMedial_L_Z, weight)","(right elbow_L_Z, right armBackMedial_L_Z, R2)","(right armFrontRadius_L_X, right armFrontUlna_L_X, weight)","(right armFrontRadius_L_X, right armFrontUlna_L_X, R2)","(right armFrontRadius_L_Y, right armFrontUlna_L_Y, weight)","(right armFrontRadius_L_Y, right armFrontUlna_L_Y, R2)","(right armBackLateral_L_Y, right armBackMedial_L_Y, weight)","(right armBackLateral_L_Y, right armBackMedial_L_Y, R2)","(right armBackLateral_L_Z, right armBackMedial_L_Z, weight)","(right armBackLateral_L_Z, right armBackMedial_L_Z, R2)"
0,0,0.043573,-0.000113,0.000042,-0.000050,-0.000154,0.000056,-0.000058,-0.000187,0.000059,...,1.046985,0.948629,1.407863,0.739600,1.177225,0.700637,0.954074,0.802546,0.975077,0.795281
1,10,0.065428,0.000134,-0.000151,0.000298,0.000172,-0.000107,0.000283,0.000174,-0.000063,...,0.710727,0.652395,-0.243486,-0.013944,0.491706,0.159646,0.653908,0.454045,0.381446,0.236318
2,1,0.041300,-0.000006,0.000038,-0.000017,-0.000020,0.000072,-0.000026,-0.000048,0.000102,...,0.964407,0.808402,1.728578,0.255870,0.617406,0.356758,0.959694,0.734347,0.603820,0.345430
3,2,0.041766,-0.000012,0.000007,-0.000018,-0.000017,0.000008,-0.000017,-0.000019,0.000010,...,0.987411,0.837402,1.481620,0.362022,0.640675,0.402472,0.925224,0.777445,0.667550,0.413517
4,3,0.043742,-0.000003,0.000023,-0.000019,-0.000004,0.000030,-0.000020,-0.000007,0.000034,...,0.884354,0.822128,1.221378,0.219664,0.845569,0.702623,0.861597,0.699333,0.592471,0.436997
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270,5,0.037364,-0.000016,-0.000022,0.000021,-0.000015,-0.000020,0.000022,-0.000014,-0.000020,...,0.913812,0.785826,0.121676,0.005792,0.688098,0.611429,0.885654,0.776612,0.552252,0.323540
271,6,0.036560,-0.000100,-0.000086,0.000025,-0.000108,-0.000120,0.000039,-0.000107,-0.000102,...,1.004507,0.903829,0.125928,0.010575,0.478454,0.318200,0.753578,0.708822,0.812723,0.627040
272,7,0.038263,0.000014,0.000010,-0.000039,0.000010,0.000017,-0.000053,0.000007,0.000022,...,1.018505,0.913061,-0.247921,-0.057200,0.345037,0.141220,0.726069,0.637005,0.848884,0.658255
273,8,0.036479,-0.000035,-0.000024,-0.000003,-0.000032,-0.000024,0.000002,-0.000029,-0.000022,...,0.987760,0.893487,-0.209472,-0.033596,0.280504,0.073141,0.722669,0.630696,0.774660,0.595158


In [26]:
y = feature_frame.fillna(0).label
X = feature_frame.fillna(0).drop(['label'], axis = 1)

In [76]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [77]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
clf = LinearDiscriminantAnalysis()
clf.fit(X_train, y_train)
#clf.scalings_
clf.score(X_test,y_test)

c:\users\chenson\appdata\local\programs\python\python37\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


0.8909090909090909